In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

c:\Users\heegyukim\Desktop\project\latent-diffusion-lm


In [2]:
from src.task.sgpt import SGPTTask
from coop.models.sgpt import SentenceGPT
import torch
from datasets import load_dataset
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra

In [3]:
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
initialize("../config/")
config = compose("sgpt")
config

{'optimizer': {'cls': 'adam', 'learning_rate': 0.0001}, 'logger': {'name': 'wandb'}, 'dataset': {'train': ['heegyu/aihub_daily_conv_2022'], 'test': ['heegyu/aihub_daily_conv_2022']}, 'model': {'model': 'sgpt-tiny.json', 'autoencoder': 'checkpoint/optimus-v2-44M/optimus-v2-tiny-vae.ckpt', 'latent_dim': 512, 'free_bit': 0.5, 'max_seq_len': 16, 'is_vae': False}, 'trainer': {'train_batch_size': 8, 'train_epochs': 20, 'shuffle': False, 'eval_batch_size': 8, 'eval_strategy': 'epoch', 'gradient_clip_val': 1.0, 'weight_clipping': 0.1, 'project': 'sgpt', 'run_name': 'tiny-opt44M', 'num_sanity_val_steps': 0, 'limit_train_batches': 10, 'limit_val_batches': 1}}

In [4]:
sgpt = SGPTTask(config).cuda()

In [6]:
batch = next(iter(sgpt.train_dataloader()))
batch[0]

Using custom data configuration heegyu--aihub_daily_conv_2022-f0169ea55f3eccaa
Reusing dataset parquet (C:\Users\heegyukim\.cache\huggingface\datasets\parquet\heegyu--aihub_daily_conv_2022-f0169ea55f3eccaa\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


{'id': 140329,
 'depth': 18,
 'speakers': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
 'utterances': ['**는 게임 좋아하니?',
  '게임 어떤거 좋아해?',
  '게임 잘 못해서,',
  '우린 완전 게임 좋아해',
  '나는 별로 안 좋아해',
  '어떤 게임 주로 하세요?',
  '그럼... 보드 게임은 어대',
  '게임은 너무 시간낭비야',
  '보드게임은 단합하기 좋죵',
  '우리는 음 맞고도 치고 루미 큐브도 하고',
  '게임 할 시간에 난 책을 읽어',
  '요즘 보드게임 카페 잘 되어 있잖아용',
  '너 책 2년 동안 한권도 안 읽었...',
  '책을 읽는 게 좋아',
  '와우 엄청난 지식인',
  '보드 게임 카페가 있어?',
  '책은 정말 너무신기해',
  '넵 요즘 애들 다 보드게임 카페가용 키키'],
 'eous': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
loss, output = sgpt.step(batch, 0)

In [7]:
loss

tensor(42.9597, device='cuda:0', grad_fn=<DivBackward0>)

In [7]:
item, sents = sgpt._item_for_train(batch[0]["utterances"][:4])
item

{'inputs': Normal(loc: torch.Size([16, 512]), scale: torch.Size([16, 512])),
 'attention_mask': tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 'labels': Normal(loc: torch.Size([16, 512]), scale: torch.Size([16, 512]))}

In [13]:
x, y = item["inputs"], item["labels"]
# x.loc - y.loc
sgpt.model.kldiv_loss(x, y, attention_mask=item["attention_mask"], use_free_bit=False)

tensor(0., device='cuda:0')

In [27]:
hidden = sgpt.model.model(
    inputs_embeds = item["inputs"].rsample().unsqueeze(0),
    attention_mask = item["attention_mask"].unsqueeze(0)
).last_hidden_state
x = sgpt.model.reparameterize(hidden)
hidden.shape, x

TypeError: tuple indices must be integers or slices, not str